In [1]:
import sys
sys.path.append('/Users/apple/Documents/ML_Project/ML - 2.1/SVR+')
from SV_Quad import *

In [31]:
import datetime
import pandas as pd
def svr_plus(X_train, X_test, 
             Y_train, Y_test, 
             X_star=None, 
             Parameters={'C': 10,
                         'gamma_corSpace': 10,
                         'gamma_rbf': 1,
                         'gamma_rbf_corSpace': 10,
                         'epsilon': 0.1,
                         'tol': 0.0001}, 
             return_y_pred=False, 
             return_mse=False,
             plot_predict=True,             
             Y_scaler=None):
    print('\n Training... ', datetime.datetime.now())
    K = rbf_kernel(X_train, Y=None, gamma = Parameters['gamma_rbf'])
    reg = SVR_Plus_Quad(X_train, X_star, Y_train, K ,Parameters)
    Kern = rbf_kernel(X_train, X_test, gamma=Parameters['gamma_rbf'])
    
    print('\n Predict... ', datetime.datetime.now())
    Y_preds = np.dot(np.transpose((reg['alpha_star'][:, 0] - reg['alpha'][:, 0])[:, None]) , Kern) + reg['bias']
    test_MSE = mean_squared_error(Y_test, np.squeeze(Y_preds))
    print ('\n Mean Squared Error: ' + str(test_MSE))
    
    if plot_predict:
        if Y_scaler is not None:
            df = pd.concat([pd.Series(Y_scaler.inverse_transform(Y_test).reshape(-1,)), 
                            pd.Series(Y_scaler.inverse_transform(Y_preds).reshape(-1,))], axis=1)
            df.columns = ['test','pred']
            df.plot(figsize=(10,4), title='MSE:{}'.
                 format(round(test_MSE,4)))
        else:
            df = pd.concat([pd.Series(Y_test).reshape(-1,), 
                            pd.Series(Y_preds).reshape(-1,)], axis=1)
            df.columns = ['test','pred']
            df.plot(figsize=(10,4), title='MSE:{}'.
                    format(round(test_MSE,4)))
    if (return_y_pred) & (return_mse):
        return pd.Series(Y_preds), test_MSE
    if return_y_pred:
        return y_pred
    if return_mse:
        return test_MSE

In [32]:
import numpy as np
from utils import get_data, get_data2

X_train, X_test, Y_train, Y_test ,Y_scaler= get_data(
    hour_num=1, transform='sin+cos',
    train_index=[6426,6527],
    test_index=[14389,15390],
    return_y_scaler=True)

X_train = np.array(X_train)
X_test = np.array(X_test)
Y_train = np.array(Y_train).reshape(-1, 1)
Y_test = np.array(Y_test).reshape(-1, 1)
X_star = Y_train
Parameters = {'C': 10,
              'gamma_corSpace': 10,
              'gamma_rbf': 1,
              'gamma_rbf_corSpace': 10,
              'epsilon': 0.1,
              'tol': 0.0001}

svr_plus(X_train, X_test, Y_train, Y_test, X_star, Parameters)

get_data(hour_num=1, transform='sin+cos', drop_time=True, scale=True)

Input space: Index(['wind_speed', 'sin(wd)', 'cos(wd)', 'wind_speed-1', 'sin(wd)-1',
       'cos(wd)-1', 'wind_power-1'],
      dtype='object')
train index: [6426, 6527] train_len: 100
test index: [14389, 15390] test_len: 1000

 Training...  2020-03-09 00:30:05.355759

 Predict...  2020-03-09 00:30:05.436092

 Mean Squared Error: 0.37187823496075273


Exception: Data must be 1-dimensional